In [1]:
import torch
from enformer_pytorch import Enformer

/projects/ps-renlab2/sunan/anaconda3/envs/enformer_pytorch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = Enformer.from_hparams(
    dim = 1536,
    depth = 11,
    heads = 8,
    output_heads = dict(human = 5313, mouse = 1643),
    target_length = 896,
)
    
seq = torch.randint(0, 5, (1, 196_608)) # for ACGTN, in that order (-1 for padding)
output = model(seq)

output['human'] # (1, 896, 5313)
output['mouse'] # (1, 896, 1643)

tensor([[[0.7227, 0.5711, 0.6251,  ..., 0.6369, 0.7891, 0.6241],
         [0.6697, 0.5736, 0.6709,  ..., 0.6348, 0.7374, 0.6948],
         [0.5989, 0.6619, 0.8082,  ..., 0.7603, 0.6847, 0.7742],
         ...,
         [0.6839, 0.6627, 0.7101,  ..., 0.6694, 0.7648, 0.6375],
         [0.6704, 0.5838, 0.7504,  ..., 0.6395, 0.7257, 0.6479],
         [0.6304, 0.6314, 0.6335,  ..., 0.7391, 0.7876, 0.8259]]],
       grad_fn=<SoftplusBackward0>)

In [8]:
from enformer_pytorch import Enformer, seq_indices_to_one_hot
model = Enformer.from_hparams(
    dim = 1536,
    depth = 11,
    heads = 8,
    output_heads = dict(human = 5313, mouse = 1643),
    target_length = 896,
)

seq = torch.randint(0, 5, (1, 196_608))
one_hot = seq_indices_to_one_hot(seq)

output = model(one_hot)

output['human'] # (1, 896, 5313)
output['mouse'] # (1, 896, 1643)


tensor([[[0.6487, 0.7115, 0.6171,  ..., 0.6917, 0.7177, 0.7240],
         [0.5758, 0.7815, 0.6534,  ..., 0.8020, 0.8190, 0.7712],
         [0.6355, 0.6589, 0.6536,  ..., 0.6495, 0.7747, 0.8103],
         ...,
         [0.6162, 0.7984, 0.7043,  ..., 0.6981, 0.6390, 0.7565],
         [0.5688, 0.8153, 0.7222,  ..., 0.6674, 0.7446, 0.7474],
         [0.6562, 0.7683, 0.6175,  ..., 0.7304, 0.6856, 0.7335]]],
       grad_fn=<SoftplusBackward0>)

In [9]:
import torch
from enformer_pytorch import Enformer, seq_indices_to_one_hot

model = Enformer.from_hparams(
    dim = 1536,
    depth = 11,
    heads = 8,
    output_heads = dict(human = 5313, mouse = 1643),
    target_length = 896,
)

seq = torch.randint(0, 5, (1, 196_608))
one_hot = seq_indices_to_one_hot(seq)

output, embeddings = model(one_hot, return_embeddings = True)

embeddings # (1, 896, 3072)

tensor([[[-0.1624, -0.0074, -0.1611,  ...,  0.0215, -0.1547,  0.1957],
         [-0.0485,  0.7589, -0.1491,  ..., -0.0914,  0.2149, -0.0246],
         [-0.0544, -0.1033, -0.1592,  ...,  0.0000,  0.1360,  0.2412],
         ...,
         [-0.1632,  0.1100, -0.1588,  ...,  0.3035,  0.0000, -0.0145],
         [-0.1434, -0.0886, -0.1269,  ..., -0.1216,  0.1450, -0.0230],
         [-0.0593,  0.4288, -0.1504,  ...,  0.2306, -0.0000,  0.0000]]],
       grad_fn=<MulBackward0>)

In [16]:
len(seq[0])

196608

In [17]:
import torch
from enformer_pytorch import Enformer, seq_indices_to_one_hot

model = Enformer.from_hparams(
    dim = 1536,
    depth = 11,
    heads = 8,
    output_heads = dict(human = 5313, mouse = 1643),
    target_length = 200,
).cuda()

seq = torch.randint(0, 5, (196_608 // 2,)).cuda()
target = torch.randn(200, 5313).cuda()

loss = model(
    seq,
    head = 'human',
    target = target
)

loss.backward()

# after much training

corr_coef = model(
    seq,
    head = 'human',
    target = target,
    return_corr_coef = True
)

corr_coef # pearson R, used as a metric in the paper

tensor(0.0008, device='cuda:0', grad_fn=<MeanBackward1>)

In [19]:
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam

model = Enformer.from_hparams(
    dim = 1536,
    depth = 11,
    heads = 8,
    output_heads = dict(human = 5313, mouse = 1643),
    target_length = 200,
).cuda()

seq = torch.randint(0, 5, (196_608 // 2,)).cuda()
target = torch.randn(200, 5313).cuda()

loss = model(
    seq,
    head = 'human',
    target = target
)

loss.backward()

# after much training
num_epochs = 5
train_seqs = torch.randint(0, 5, (100, 196_608 // 2))
train_targets = torch.randn(100, 200, 5313)

train_dataset = TensorDataset(train_seqs, train_targets)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

optimizer = Adam(model.parameters(), lr=1e-4)

for epoch in range(num_epochs):
    for seq, target in train_loader:
        seq, target = seq.cuda(), target.cuda()

        # Forward pass
        loss = model(seq, head="human", target=target)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {loss.item():.4f}")




corr_coef = model(
    seq,
    head = 'human',
    target = target,
    return_corr_coef = True
)

corr_coef # pearson R, used as a metric in the paper

Epoch 1, Loss: 0.2911
Epoch 2, Loss: 0.0087
Epoch 3, Loss: -0.0104
Epoch 4, Loss: -0.0541
Epoch 5, Loss: -0.1054


tensor([0.0055, 0.0075, 0.0065, 0.0064], device='cuda:0',
       grad_fn=<MeanBackward1>)

In [2]:
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
from enformer_pytorch import Enformer
import torch

# Define the model
model = Enformer.from_hparams(
    dim=1536,
    depth=11,
    heads=8,
    output_heads=dict(human=5313, mouse=1643),
    target_length=200,
).cuda()

# Training configurations
num_epochs = 5
batch_size = 8
train_seqs = torch.randint(0, 5, (100, 196_608 // 2))  # 100 sequences
train_targets = torch.randn(100, 200, 5313)  # corresponding targets

# Dataset and DataLoader
train_dataset = TensorDataset(train_seqs, train_targets)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Optimizer
optimizer = Adam(model.parameters(), lr=1e-4)

# Calculate steps per epoch
steps_per_epoch = len(train_loader)
# Training loop
for epoch in range(num_epochs):
    total_loss = 0.0
    for step, (seq, target) in enumerate(train_loader, start=1):
        seq, target = seq.cuda(), target.cuda()

        # Forward pass
        loss = model(seq, head="human", target=target)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Log every 10 steps
        if step % 10 == 0 or step == steps_per_epoch:
            print(f"Epoch {epoch + 1}/{num_epochs}, Step {step}/{steps_per_epoch}, "
                  f"Loss: {loss.item():.4f}")

    # Average loss for the epoch
    avg_loss = total_loss / steps_per_epoch
    print(f"Epoch {epoch + 1} completed. Average Loss: {avg_loss:.4f}")

# Evaluate Pearson Correlation Coefficient
seq = torch.randint(0, 5, (196_608 // 2,)).cuda()
target = torch.randn(200, 5313).cuda()
corr_coef = model(seq, head="human", target=target, return_corr_coef=True)
print(f"Pearson Correlation Coefficient: {corr_coef}")

/projects/ps-renlab2/sunan/anaconda3/envs/enformer_pytorch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch 1/5, Step 10/13, Loss: 0.4358
Epoch 1/5, Step 13/13, Loss: 0.2922
Epoch 1 completed. Average Loss: 0.5321
Epoch 2/5, Step 10/13, Loss: 0.0194
Epoch 2/5, Step 13/13, Loss: 0.0103
Epoch 2 completed. Average Loss: 0.0828
Epoch 3/5, Step 10/13, Loss: 0.0027
Epoch 3/5, Step 13/13, Loss: 0.0018
Epoch 3 completed. Average Loss: 0.0023
Epoch 4/5, Step 10/13, Loss: -0.0088
Epoch 4/5, Step 13/13, Loss: -0.0252
Epoch 4 completed. Average Loss: -0.0082
Epoch 5/5, Step 10/13, Loss: -0.0288
Epoch 5/5, Step 13/13, Loss: -0.0260
Epoch 5 completed. Average Loss: -0.0290
Pearson Correlation Coefficient: 0.0011403672397136688
